# Find optimal TTNDO structure for a Simple Open Quantum System

We simulate a TFI model
$$
H = \sum_{i=1}^{L-1} X_iX_{i+1} + \sum_{i=1}^L Z_i
$$
with local relaxation $\sigma^-_i$ and dephasing $Z_i$.

In [1]:
import numpy as np
# Parameters for simulating a 1D transverse field Ising model with dissipation

# length: Number of spins in the chain
length = 11
# ext_magn: External magnetic field strength in z-direction
ext_magn = 0.5
# coupling: Nearest-neighbor coupling strength for XX interactions
coupling = 1.0 
# relaxation_rate: Local amplitude damping (relaxation) rate
relaxation_rate = 0.1
# dephasing_rate: Local dephasing rate
dephasing_rate = 0.1
# time_step_size: Size of time steps for evolution
time_step_size = 0.01
# final_time: Total simulation time
final_time = 0.4
# Initial local states
phys_tensor = np.array([1, 0])


## Exact Solution with Qutip

In [ ]:
from TFI_1D_Qutip import TFI_1D_Qutip

tfi_system = TFI_1D_Qutip(L=length, 
                                          coupling=coupling, 
                                          ext_magn=ext_magn, 
                                          periodic=False, 
                                          use_single_precision = False)
# Initial State
initial_state = tfi_system.uniform_product_state(phys_tensor)
# Build Jump Operators
jump_operators = tfi_system.build_jump_operators(relaxation_rate, dephasing_rate)
# Observables
energy_op = tfi_system.hamiltonian
total_mag_z = sum(tfi_system.sz_ops)
total_mag_z /= length
e_ops = [total_mag_z , energy_op]
# Evolve the System 
options = {'method': 'bdf', 
            'atol': 1e-8,
            'rtol': 1e-6,
            'nsteps': 1000 }         
result, solve_time = tfi_system.evolve_system_lindblad(initial_state=initial_state,
                                                        final_time=final_time,
                                                        time_step_size=time_step_size,
                                                        jump_operators=jump_operators,
                                                        e_ops=e_ops,
                                                        options=options)
# Collect the results
exact_magns = np.array(result.expect[:-1])
exact_energy = np.array(result.expect[-1])
print(f"results shape: {np.array(result.expect).shape}")

In [2]:
exact_magns = np.array([[1.        , 0.99963692, 0.99855224, 0.9967517 , 0.99424297,
        0.99103513, 0.98713834, 0.98256389, 0.97732454, 0.9714337 ,
        0.96490609, 0.95775759, 0.95000492, 0.94166607, 0.93275977,
        0.92330576, 0.91332446, 0.90283708, 0.89186556, 0.8804325 ,
        0.86856109, 0.85627511, 0.84359884, 0.83055695, 0.81717458,
        0.80347713, 0.78949029, 0.77523999, 0.76075227, 0.74605329,
        0.73116928, 0.71612641, 0.7009508 , 0.68566848, 0.67030526,
        0.65488672, 0.63943822, 0.62398472, 0.60855083, 0.59316077,
        0.57783824]])

In [6]:
exact_energy = np.array([-5.5       , -5.49999698, -5.49997778, -5.49992745, -5.49983027,
       -5.49967088, -5.49943432, -5.49910596, -5.49867159, -5.49811723,
       -5.49742931, -5.49659472, -5.49560069, -5.49443499, -5.49308583,
       -5.4915419 , -5.48979236, -5.48782693, -5.48563579, -5.48320969,
       -5.48053994, -5.47761839, -5.47443745, -5.47099014, -5.46727003,
       -5.46327129, -5.45898867, -5.45441753, -5.44955381, -5.44439404,
       -5.43893536, -5.43317547, -5.42711269, -5.42074589, -5.41407455,
       -5.40709867, -5.39981885, -5.39223621, -5.38435244, -5.37616972,
       -5.36769078])

## Tree Tensor Network Simulation

In [2]:
from fractions import Fraction

from numpy import ndarray
from typing import Union
from pytreenet.special_ttn.binary import generate_binary_ttns
from pytreenet.ttns.ttndo import (SymmetricTTNDO,
                                  IntertwinedTTNDO,
                                  from_ttns_symmetric,
                                  from_ttns_fully_intertwined,
                                  from_ttns_physically_intertwined)
from pytreenet.operators.models import (ising_model,
                                        flipped_ising_model,
                                        local_magnetisation,)
from pytreenet.operators.sim_operators import single_site_operators
from pytreenet.operators.common_operators import pauli_matrices
from pytreenet.operators.tensorproduct import TensorProduct
from pytreenet.operators.lindbladian import generate_lindbladian
from pytreenet.ttno.ttno_class import TreeTensorNetworkOperator
from pytreenet.operators.hamiltonian import Hamiltonian
from pytreenet.operators.common_operators import (bosonic_operators,pauli_matrices)
from pytreenet.ttns.ttns import TreeTensorNetworkState
from pytreenet.time_evolution.bug import BUG, BUGConfig

In [3]:
def open_ising_model_ttno(length: int,
                          ttndo: Union[SymmetricTTNDO, IntertwinedTTNDO],
                          ext_magn: float,
                          coupling: float,
                          relaxation_rate: float,
                          dephasing_rate: float,
                          flipped: bool = False
                          ) -> tuple[TreeTensorNetworkOperator, Hamiltonian]:
    """
    Generates the Hamiltonian for the open Ising model.

    Additionally to the usual TFI model two jump operators act on every site
    separately. The first jump operator is relaxation and the second is 
    dephasing.

    Args:
        length (int): The length of the system.
        ttndo (SymmetricTTNDO): The TTNDO of the system.
        ext_magn (float): The external magnetic field.
        coupling (float): The coupling strength.
        relaxation_rate (float): The relaxation rate.
        dephasing_rate (float): The dephasing rate.
        flipped (bool): Whether to use the flipped Ising model.

    Returns:
        TreeTensorNetworkOperator: The TTNO.
    
    """
    node_identifiers = phys_node_identifiers(length)
    nn_pairs = [(node_identifiers[i], node_identifiers[i+1])
                for i in range(length-1)]
    if flipped:
        ham = flipped_ising_model(nn_pairs,
                                  ext_magn,
                                  factor=coupling)
    else:
        ham = ising_model(nn_pairs,
                          ext_magn,
                          factor=coupling)
    jump_ops, conversion_dict, coeff_dict = jump_operators(length,
                                                          relaxation_rate,
                                                          dephasing_rate)
    lindbladian = generate_lindbladian(ham,
                                        jump_ops,
                                        conversion_dict,
                                        coeff_dict)
    ttno = TreeTensorNetworkOperator.from_hamiltonian(lindbladian,
                                                      ttndo)
    return ttno, ham

def jump_operators(length: int,
                   relaxation_rate: float,
                   dephasing_rate:float
                   ) -> tuple[list[tuple[Fraction,str,TensorProduct]], dict[str,ndarray], dict[str,complex]]:
    """
    Generates the jump operators for the open Ising model.

    Args:
        length (int): The length of the system.
        relaxation_rate (float): The relaxation rate.
        dephasing_rate (float): The dephasing rate.

    Returns:
        list[tuple[Fraction,str,TensorProduct]]: The jump operator terms.
        dict[str,ndarray]: The jump operator conversion dictionary.
        dict[str,float]: The jump operator coefficient mapping
    
    """
    node_identifiers = phys_node_identifiers(length)
    # Create relaxation jump operators
    rel_fac_name = "relaxation_rate"
    factor = (Fraction(1), rel_fac_name)
    relax_name = "sigma_-"
    relaxation_ops = single_site_operators(relax_name,
                                           node_identifiers,
                                           factor)
    # Create dephasing jump operators
    deph_fac_name = "dephasing_rate"
    factor = (Fraction(1), deph_fac_name)
    deph_name = "Z"
    dephasing_ops = single_site_operators(deph_name,
                                          node_identifiers,
                                          factor)
    # Combine the jump operators
    jump_ops = list(relaxation_ops.values())
    jump_ops.extend(list(dephasing_ops.values()))
    # Create the jump operator matrix mapping
    conversion_dict = {relax_name: bosonic_operators()[1],
                       deph_name: pauli_matrices()[2]}
    # Create the jump operator coefficient mapping
    coeff_dict = {rel_fac_name: complex(relaxation_rate),
                  deph_fac_name: complex(dephasing_rate)}
    return jump_ops, conversion_dict, coeff_dict

def phys_node_identifiers(length: int) -> list[str]:
    """
    Generates the node identifiers for the physical nodes.

    Args:
        length (int): The length of the system.
    
    Returns:
        list[str]: The node identifiers.
    """
    return [f"qubit{i}" for i in range(length)]

def open_ising_operators(length: int,
                         ttns: TreeTensorNetworkState,
                         ising_ham: Hamiltonian
                         ) -> dict:
    """
    Generates the operators to be evaluated for the open Ising model.
    
    Args:
        length (int): The length of the system.
        ttns (TreeTensorNetworkState): The TTN state of the system.
        ising_ham (Hamiltonian): The Ising Hamiltonian.
    
    Returns:
        dict: The operators.

    """
    node_identifiers = phys_node_identifiers(length)
    ops = {key: tup[2]
           for key, tup in local_magnetisation(node_identifiers).items()}
    ttno = TreeTensorNetworkOperator.from_hamiltonian(ising_ham,
                                                      ttns)
    ops["energy"] = ttno
    return ops

In [4]:
import time 
from ttn_vis import (visualize_binary_ttns,
                    visualize_symmetric_ttndo,
                    visualize_intertwined_ttndo)


def run_simulation(ttndo_type_name,
                   initial_bond_dim,
                   max_bond_dim,
                   rel_tol = 1e-4,
                   total_tol = 1e-3,
                   depth = 4,
                   visualize = False):
    
    print(f"{ttndo_type_name} with depth = {depth}")
    
    # Generate binary tree tensor network state
    ttns = generate_binary_ttns(num_phys=length,
                                bond_dim=initial_bond_dim,
                                phys_tensor=phys_tensor,
                                depth=depth)
    if visualize :
        ttns_virtual_count = sum(1 for node_id in ttns.nodes if not ("qubit" in node_id or "site" in node_id))
        visualize_binary_ttns(ttns, 
                              title=f'Binary TTNS Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={ttns_virtual_count})',
                              layout_type="hierarchical",
        simplified_labels=True)
        visualize_binary_ttns(ttns, 
                            title=f'Binary TTNS Radial (Depth={depth}, #qubits={length}, #Virtual nodes={ttns_virtual_count})',
                            layout_type="radial",
                            simplified_labels=True)
    if ttndo_type_name == "symmetric" :
        ttndo = from_ttns_symmetric(ttns)
        if visualize :
            # Count ket virtual nodes and double it
            sym_virtual_count = ttns_virtual_count * 2 + 1
            visualize_symmetric_ttndo(ttndo, 
                                      title=f'Symmetric TTNDO (Depth={depth}, #qubits={length}, #Virtual nodes={sym_virtual_count})',
                                      simplified_labels=True)
    elif ttndo_type_name == "fully_intertwined" :
        ttndo = from_ttns_fully_intertwined(ttns, initial_bond_dim, phys_tensor) 
        if visualize :
           # Count ket virtual nodes and double it
            int_ket_virtual_count = sum(1 for node_id in ttndo.nodes 
                                        if "_ket" in node_id and not ("qubit" in node_id or "site" in node_id))
            int_virtual_count = int_ket_virtual_count * 2            
            visualize_intertwined_ttndo(ttndo, 
                                        title=f'Full Intertwined TTNDO Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
                                        layout_type="hierarchical",
                                        simplified_labels=True)
            visualize_intertwined_ttndo(ttndo, 
                                        title=f'Full Intertwined TTNDO Radial (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
                                        layout_type="radial",
                                        simplified_labels=True)
    elif ttndo_type_name == "physically_intertwined" :
        ttndo = from_ttns_physically_intertwined(ttns, initial_bond_dim, phys_tensor) 
        if visualize :
            visualize_intertwined_ttndo(ttndo, 
                                        title=f'Physical Intertwined TTNDO Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
                                        layout_type="hierarchical",
                                        simplified_labels=True)

            # Intertwined TTNDO - Radial
            print("Creating Intertwined TTNDO Radial plot...")
            visualize_intertwined_ttndo(ttndo, 
                                        title=f'Physical Intertwined TTNDO Radial (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
                                        layout_type="radial",
                                        simplified_labels=True)

    # Define Ising model
    lindbladian_ttno, ising_ham = open_ising_model_ttno(length,
                                                        ttndo,
                                                        ext_magn,
                                                        coupling,
                                                        relaxation_rate,
                                                        dephasing_rate)
    
    # Observables
    operators = open_ising_operators(length,
                                     ttns,
                                     ising_ham)
    
    # BUG simulation
    config = BUGConfig(max_bond_dim=max_bond_dim,
                        rel_tol=rel_tol,
                        total_tol=total_tol,
                        record_bond_dim=True)
    from patch_ttndo_bug import fix_ttndo_bug, restore_original
    original = fix_ttndo_bug()
    bug = BUG(ttndo,
              lindbladian_ttno,
              time_step_size,
              final_time,
              operators,
              config=config)
    start_time = time.time()
    bug.run()
    CPU_time = time.time() - start_time
    restore_original(original)

    # Collect the results
    bug_magns = np.sum(bug._results[:-2], axis=0) / length
    bug_energy = bug._results[-2]

    print(f"CPU_time = {CPU_time:.2f} s")
        
    return bug_magns, bug_energy



def Calculate_absolute_error_and_save(results, 
                                      bug_magns, 
                                      bug_energy, 
                                      ttndo_type_name, 
                                      depth, 
                                      exact_magns, 
                                      exact_energy):
    if ttndo_type_name not in results:
        results[ttndo_type_name] = {}
    
    magn_error = np.abs(bug_magns - exact_magns)
    magn_error = magn_error.reshape(-1)
    energy_error = np.abs(bug_energy - exact_energy)
    energy_error = energy_error.reshape(-1)
    
    results[ttndo_type_name][depth] = {"magn_error": magn_error,
                                       "energy_error": energy_error}
    

# Optimal TTNDO 

we examine three distinct TTNDO structural at various tree depths to identify optimal configurations for simulating open quantum systems.

### TTNDO Structures
1. **Symmetric TTNDO**
2. **Fully Intertwined TTNDO**
3. **Physically Intertwined TTNDO**

For each TTNDO type $\theta \in \{\text{symmetric, fully\_intertwined, physically\_intertwined}\}$ and depth $d \in \{\text{depths}\}$, we try different initial_bond_dim and max_bond_dim
to have a simulation with CPU_time = 

1. $$\{\langle\hat{\sigma}_x\rangle(t), E(t)\} = \text{run\_simulation}(d, \theta, \mathcal{D}_{\text{init}}, \mathcal{D}_{\text{max}})$$
   
2. Calculate error metrics against exact solution:
$$\epsilon_{\text{magn}} = |\langle\hat{\sigma}_z\rangle - \langle\hat{\sigma}_z\rangle_{\text{exact}}|$$
$$\epsilon_{\text{energy}} = \hat{E} - \hat{E}_{\text{exact}}|$$
Calculate_absolute_error_and_save:
3. results[θ][d] = {
   "magn_error": ϵ_magn,
   "energy_error": ϵ_energy}


In [2]:
results = {}

In [ ]:
depth = 4

initial_bond_dim = 10
max_bond_dim = 25
rel_tol = 1e-4
total_tol = 1e-4

bug_magns, bug_energy = run_simulation(ttndo_type_name = "symmetric",initial_bond_dim = initial_bond_dim,max_bond_dim = max_bond_dim,rel_tol = rel_tol,total_tol = total_tol,depth = depth,
                                       visualize= False)
Calculate_absolute_error_and_save(results = results, bug_magns = bug_magns, bug_energy= bug_energy, ttndo_type_name = "symmetric", depth = depth, exact_magns = exact_magns, exact_energy = exact_energy)

In [ ]:
depth = 4

initial_bond_dim = 12
max_bond_dim = 25
rel_tol = 1e-6
total_tol = 1e-6


bug_magns, bug_energy = run_simulation(ttndo_type_name = "fully_intertwined",initial_bond_dim = initial_bond_dim,max_bond_dim = max_bond_dim,rel_tol = rel_tol,total_tol = total_tol,depth = depth,
                                       visualize= False)
Calculate_absolute_error_and_save(results = results, bug_magns = bug_magns, bug_energy= bug_energy, ttndo_type_name = "fully_intertwined", depth = depth, exact_magns = exact_magns, exact_energy = exact_energy)

In [5]:
depth = 4

initial_bond_dim = 10
max_bond_dim = 20
rel_tol = 1e-5
total_tol = 1e-5


bug_magns, bug_energy = run_simulation(ttndo_type_name = "physically_intertwined",initial_bond_dim = initial_bond_dim,max_bond_dim = max_bond_dim,rel_tol = rel_tol,total_tol = total_tol,depth = depth,
                                       visualize= False)
Calculate_absolute_error_and_save(results = results, bug_magns = bug_magns, bug_energy= bug_energy, ttndo_type_name = "physically_intertwined", depth = depth, exact_magns = exact_magns, exact_energy = exact_energy)

physically_intertwined with depth = 4
Applied patch to fix TTNDO-BUG shape mismatch issues
Restored original pull_tensor_from_different_ttn function


 73%|███████▎  | 30/41 [00:12<00:04,  2.33it/s]


NotCompatibleException: Shapes of the tensor and the node do not match!

In [ ]:
def plot_ttndo_errors(results, times):
    """
    Plot error metrics for all TTNDO types and depths, including time-series and accumulated errors.
    
    Parameters:
    -----------
    results : dict
        Nested dictionary with structure results[ttndo_type][depth] = {"magn_error": error_array, "energy_error": error_array}
    times : array-like
        Time points corresponding to the error data points
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    # Define line styles and markers for different depths
    depth_styles = {
        1: {'linestyle': '-', 'marker': 'o'},
        2: {'linestyle': '--', 'marker': 's'},
        3: {'linestyle': ':', 'marker': '^'},
        4: {'linestyle': '-.', 'marker': 'x'},
        5: {'linestyle': (0, (3, 1, 1, 1)), 'marker': 'd'},  # Dash-dot-dot
        6: {'linestyle': (0, (3, 5, 1, 5)), 'marker': '*'},  # Sparse dash-dot
    }
    
    # Define colors for different TTNDO types
    ttndo_colors = {
        'symmetric': 'blue',
        'fully_intertwined': 'red',
        'physically_intertwined': 'green',
    }
    
    # Create figure for magnetization error
    plt.figure(figsize=(12, 8))
    
    # Loop through each TTNDO type and depth
    for ttndo_type, depths in results.items():
        for depth, error_data in depths.items():
            # Get the correct style for this depth
            style = depth_styles.get(depth, {'linestyle': '-', 'marker': '.'})
            color = ttndo_colors.get(ttndo_type, 'black')
            
            # Plot magnetization error
            plt.semilogy(times, error_data['magn_error'], 
                       label=f"{ttndo_type} (depth {depth})",
                       color=color,
                       linestyle=style['linestyle'],
                       marker=style['marker'],
                       markersize=6,
                       markevery=max(1, len(times)//10))  # Show markers at ~10 points
    
    plt.xlabel('Time')
    plt.ylabel('Magnetization Error (log scale)')
    plt.title('Magnetization Error Comparison Across TTNDO Types and Depths')
    plt.grid(True, which='both', linestyle='--', alpha=0.5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()

    
    plt.figure(figsize=(12, 8))
    
    # Loop through each TTNDO type and depth for energy error
    for ttndo_type, depths in results.items():
        for depth, error_data in depths.items():
            # Get the correct style for this depth
            style = depth_styles.get(depth, {'linestyle': '-', 'marker': '.'})
            color = ttndo_colors.get(ttndo_type, 'black')
            
            # Plot energy error
            plt.semilogy(times, error_data['energy_error'], 
                       label=f"{ttndo_type} (depth {depth})",
                       color=color,
                       linestyle=style['linestyle'],
                       marker=style['marker'],
                       markersize=6,
                       markevery=max(1, len(times)//10))
    
    plt.xlabel('Time')
    plt.ylabel('Energy Error (log scale)')
    plt.title('Energy Error Comparison Across TTNDO Types and Depths')
    plt.grid(True, which='both', linestyle='--', alpha=0.5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    
    # Create a summary bar plot for accumulated errors
    plt.figure(figsize=(14, 10))
    
    # Extract final error values for each configuration
    ttndo_types = list(results.keys())
    all_depths = sorted(set(depth for depths in results.values() for depth in depths.keys()))
    
    # Set up bar positions
    bar_width = 0.35
    index = np.arange(len(all_depths))
    
    for i, ttndo_type in enumerate(ttndo_types):
        accumulated_magn_errors = []
        accumulated_energy_errors = []
        
        for depth in all_depths:
            if depth in results[ttndo_type]:
                # Calculate accumulated errors (area under the error curve) using trapezoid rule
                acc_magn_error = np.trapezoid(results[ttndo_type][depth]["magn_error"], times)
                acc_energy_error = np.trapezoid(results[ttndo_type][depth]["energy_error"], times)
                
                accumulated_magn_errors.append(acc_magn_error)
                accumulated_energy_errors.append(acc_energy_error)
            else:
                accumulated_magn_errors.append(np.nan)
                accumulated_energy_errors.append(np.nan)
        
        # Create the bar plots - offset for each TTNDO type
        plt.subplot(2, 1, 1)
        plt.bar(index + i*bar_width/len(ttndo_types), accumulated_magn_errors, 
                bar_width/len(ttndo_types), label=ttndo_type, 
                color=ttndo_colors.get(ttndo_type, 'black'), alpha=0.7)
        
        plt.subplot(2, 1, 2)
        plt.bar(index + i*bar_width/len(ttndo_types), accumulated_energy_errors, 
                bar_width/len(ttndo_types), label=ttndo_type, 
                color=ttndo_colors.get(ttndo_type, 'black'), alpha=0.7)
    
    # Customize the plots
    plt.subplot(2, 1, 1)
    plt.title('Accumulated Magnetization Error by TTNDO Type and Depth')
    plt.ylabel('Accumulated Magnetization Error')
    plt.xticks(index + bar_width/2, [f'Depth {d}' for d in all_depths])
    plt.yscale('log')
    plt.legend()
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    
    plt.subplot(2, 1, 2)
    plt.title('Accumulated Energy Error by TTNDO Type and Depth')
    plt.ylabel('Accumulated Energy Error')
    plt.xticks(index + bar_width/2, [f'Depth {d}' for d in all_depths])
    plt.yscale('log')
    plt

# Example usage:
times = np.arange(0, final_time + time_step_size, time_step_size)
plot_ttndo_errors(results, times)

In [ ]:
depths = [1]

for depth in depths:
    print(f"Visualizing networks: depth={depth}, sites={length}")
    
    # 1. Generate the binary TTNS
    ttns = generate_binary_ttns(num_phys=length,
                                bond_dim=initial_bond_dim,
                                phys_tensor=phys_tensor,
                                depth=depth)
    
    # Count virtual nodes in TTNS
    ttns_virtual_count = sum(1 for node_id in ttns.nodes 
                            if not ("qubit" in node_id or "site" in node_id))
    print(f"TTNS: {len(ttns.nodes)} total nodes, {ttns_virtual_count} virtual nodes")
    
    # 2. Generate the TTNDOs
    try:
        ttndo_symmetric = from_ttns_symmetric(ttns)
        
        # SYMMETRIC TTNDO: Double TTNS virtual count + 1 for ttndo_root
        sym_virtual_count = ttns_virtual_count * 2 + 1
        print(f"Symmetric TTNDO: {len(ttndo_symmetric.nodes)} total nodes, {sym_virtual_count} virtual nodes")
    except Exception as e:
        print(f"Error creating symmetric TTNDO: {e}")
        ttndo_symmetric = None
        sym_virtual_count = 0
        
    try:
        ttndo_fully_intertwined = from_ttns_fully_intertwined(ttns, bond_dim=initial_bond_dim, phys_tensor=phys_tensor)
            
        # INTERTWINED TTNDO: Count ket virtual nodes and double it
        int_ket_virtual_count = sum(1 for node_id in ttndo_fully_intertwined.nodes 
                                    if "_ket" in node_id and not ("qubit" in node_id or "site" in node_id))
        int_virtual_count = int_ket_virtual_count * 2
        print(f"Intertwined TTNDO: {len(ttndo_fully_intertwined.nodes)} total nodes, {int_virtual_count} virtual nodes")
    except Exception as e:
        print(f"Error creating intertwined TTNDO: {e}")
        ttndo_fully_intertwined = None
        int_virtual_count = 0

    try:
        ttndo_physically_intertwined = from_ttns_physically_intertwined(ttns, bond_dim=initial_bond_dim, phys_tensor=phys_tensor)
    except Exception as e:
        print(f"Error creating intertwined TTNDO: {e}")

    # 3. Now create visualizations one by one
    # Binary TTNS - Hierarchical
    print("Creating Binary TTNS Hierarchical plot...")
    visualize_binary_ttns(
        ttns, 
        title=f'Binary TTNS Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={ttns_virtual_count})',
        layout_type="hierarchical",
        simplified_labels=True
    )

    # Binary TTNS - Radial
    print("Creating Binary TTNS Radial plot...")
    visualize_binary_ttns(
        ttns, 
        title=f'Binary TTNS Radial (Depth={depth}, #qubits={length}, #Virtual nodes={ttns_virtual_count})',
        layout_type="radial",
        simplified_labels=True
    )
    
    # Symmetric TTNDO
    if ttndo_symmetric:
        print("Creating Symmetric TTNDO plot...")
        visualize_symmetric_ttndo(
            ttndo_symmetric, 
            title=f'Symmetric TTNDO (Depth={depth}, #qubits={length}, #Virtual nodes={sym_virtual_count})',
            simplified_labels=True
        )
    
    # Intertwined TTNDO - Hierarchical
    if ttndo_fully_intertwined:
        print("Creating Intertwined TTNDO Hierarchical plot...")
        visualize_intertwined_ttndo(
            ttndo_fully_intertwined, 
            title=f'Full Intertwined TTNDO Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
            layout_type="hierarchical",
            simplified_labels=True
        )
    
        # Intertwined TTNDO - Radial
        print("Creating Intertwined TTNDO Radial plot...")
        visualize_intertwined_ttndo(
            ttndo_fully_intertwined, 
            title=f'Full Intertwined TTNDO Radial (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
            layout_type="radial",
            simplified_labels=True
        )

        visualize_intertwined_ttndo(
            ttndo_physically_intertwined, 
            title=f'Physical Intertwined TTNDO Hierarchical (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
            layout_type="hierarchical",
            simplified_labels=True)

        # Intertwined TTNDO - Radial
        print("Creating Intertwined TTNDO Radial plot...")
        visualize_intertwined_ttndo(
            ttndo_physically_intertwined, 
            title=f'Physical Intertwined TTNDO Radial (Depth={depth}, #qubits={length}, #Virtual nodes={int_virtual_count})',
            layout_type="radial",
            simplified_labels=True)
        
